Mount Google Drive (optional)

In [198]:
# from google.colab import drive
# drive.mount('/content/drive')

# import os
# # os.chdir("/content/drive/MyDrive/....")    # file path
# print(os.getcwd())

# **HW2 : Decision Tree and Random Forest**
In *assignment 2*, you need to finish :

1. Basic Part : Implement a **Decision Tree** model and predict whether the patients in the validation set have diabetes
> * Step 1 : Load the input data
> * Step 2 : Calculate the Entropy and Information Gain
> * Step 3 : Find the Best Split
> * Step 4 : Split into 2 branches
> * Step 5 : Build decision tree
> * Step 6 : Save the answers from step2 to step5
> * Step 7 : Split data into training set and validation set
> * Step 8 : Train a decision tree model with training set
> * Step 9 : Predict the cases in the *validation set* by using the model trained in *Step8*
> * Step 10 : Calculate the f1-score of your predictions in *Step9*
> * Step 11 : Write the Output File

2. Advanced Part : Build a **Random Forest** model to make predictions
> * Step 1 : Load the input data
> * Step 2 : Load the test data
> * Step 3 : Build a random forest
> * Step 4 : Predict the cases in the test data by using the model trained in *Step3*
> * Step 5 : Save the predictions(from *Step 4*) in a csv file



# **Basic Part** (60%)
In this part, your need to implement a Decision Tree model by completing the following given functions.

Also, you need to run these functions with the given input variables and save the output in a csv file **hw2_basic.csv**

## Import Packages


> Note : You **cannot** import any other packages in both basic part and advanced part

In [199]:
import numpy as np
import pandas as pd
import math
import random
from numpy import sqrt
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

## Step1: Load the input data
First, load the input file **hw2_input_basic.csv**

In [200]:
input_data = pd.read_csv('hw2_input_basic.csv')
input_data

,age,bmi,gender,height,weight,glucose_apache,heart_rate_apache,resprate_apache,sodium_apache,diabetes_mellitus
0,70.0,25.984659,1,172.7,77.50,116.0,101.0,49.0,137.0,0
1,30.0,31.310368,1,170.2,90.70,71.0,39.0,33.0,144.0,0
2,54.0,24.388824,1,177.8,77.10,120.0,120.0,31.0,141.0,0
3,65.0,34.141074,0,170.2,98.90,73.0,48.0,36.0,140.0,1
4,49.0,22.564743,1,172.7,67.30,207.0,119.0,6.0,144.0,0
5,62.0,29.424010,0,154.9,70.60,113.0,60.0,32.0,137.0,0
6,85.0,27.673574,1,154.9,66.40,102.0,49.0,36.0,142.0,0
7,65.0,22.269432,1,177.8,70.40,333.0,59.0,6.0,145.0,1
8,85.0,35.879362,0,165.1,97.80,124.0,92.0,30.0,136.0,0
9,81.0,20.859375,0,160.0,53.40,136.0,118.0,52.0,138.0,0


## Global attributes
Define the global attributes
> Note : You **cannot** modify the values of these attributes we given in the basic part

In [201]:
max_depth = 2
depth = 0
min_samples_split = 2
n_features = input_data.shape[1] - 1

> You can add your own global attributes here

## Step2 : Calculate the Entropy and Information Gain 
Calculate the information gain and entropy values before separate data into left subtree and right subtree

In [202]:
def entropy(data):
    """
    This function measures the amount of uncertainty in a probability distribution
    args: 
    * data(type: DataFrame): the data you're calculating for the entropy
    return:
    * entropy_value(type: float): the data's entropy
    """

    values, counts = np.unique(data.get("diabetes_mellitus"), return_counts = True)
    probs = counts / len(data.get("diabetes_mellitus"))

    p = np.mean(data.get("diabetes_mellitus"))
    entropy_value = 0.
    for i in probs:
        if i == 1:
            i = i - 1e-9
        elif i == 0:
            i = i + 1e-9
        entropy_value -= i * math.log2(i)
    
    return entropy_value

# [Note] You have to save the value of "ans_entropy" into the output file
ans_entropy = entropy(input_data)
print("ans_entropy = ", ans_entropy)

ans_entropy =  0.9871377743721863


In [203]:
def information_gain(data, mask):
    """
    This function will calculate the information gain
    args:
    * data(type: DataFrame): the data you're calculating for the information gain
    * mask(type: Series): partition information(left/right) of current input data, 
        - boolean 1(True) represents split to left subtree
        - boolean 0(False) represents split to right subtree
    return:
    * ig(type: float): the information gain you can obtain by classify data with this given mask
    """
    left = data[mask]
    right = data[~mask]
    values, counts = np.unique(data.get("diabetes_mellitus"), return_counts = True)
    left_values, left_counts = np.unique(left.get("diabetes_mellitus"), return_counts = True)
    right_values, right_counts = np.unique(right.get("diabetes_mellitus"), return_counts = True)
    EnAfter = sum(left_counts) / sum(counts) * entropy(left) + sum(right_counts) / sum(counts) * entropy(right)
    ig = entropy(data) - EnAfter
    
    return ig

# [Note] You have to save the value of "ans_informationGain" into your output file
temp1 = np.zeros((int(input_data.shape[0]/4), 1), dtype=bool)
temp2 = np.ones(((input_data.shape[0]-int(input_data.shape[0]/4), 1)), dtype=bool)
temp_mask = np.concatenate((temp1, temp2))
df_mask = pd.DataFrame(temp_mask, columns=['mask'])
ans_informationGain = information_gain(input_data, df_mask['mask'])
print("ans_informationGain = ", ans_informationGain)

ans_informationGain =  0.08345988684807149


## Step3 : Find the Best Split
Find the best split combination, **feature** and **threshold**, by calculating the information gain


In [204]:
def find_best_split(data):
    """
    This function will find the best split combination of data
    args:
    * data(type: DataFrame): the input data
    return
    * best_ig(type: float): the best information gain you obtain
    * best_threshold(type: float): the value that splits data into 2 branches
    * best_feature(type: string): the feature that splits data into 2 branches
    """
    best_ig, best_threshold, best_feature = 0., 0., ""
    for entry in data:
        if entry == "diabetes_mellitus":
            continue
        sorted = np.sort(np.unique(data.get(entry)))
        for threshold in sorted:
            mask = np.asarray([i <= threshold for i in data[entry]])
            ig = information_gain(data, mask)
            if best_ig < ig:
                best_ig, best_threshold, best_feature = ig, threshold, entry     
    
    return best_ig, best_threshold, best_feature


# [Note] You have to save the value of "ans_ig", "ans_value", and "ans_name" into the output file
ans_ig, ans_value, ans_name = find_best_split(input_data)
print("ans_ig = ", ans_ig)
print("ans_value = ", ans_value)
print("ans_name = ", ans_name)

ans_ig =  0.35229505124460436
ans_value =  207.0
ans_name =  glucose_apache


## Step4 : Split into 2 branches
Using the best split combination you find in function *find_best_split()* to split data into Left Subtree and Right Subtree 

In [205]:
def make_partition(data, feature, threshold):
    """
    This function will split the data into 2 branches
    args:
    * data(type: DataFrame): the input data
    * feature(type: string): the attribute(column name)
    * threshold(type: float): the threshold for splitting the data
    return:
    * left(type: DataFrame): the divided data that matches(less than or equal to) the assigned feature's threshold
    * right(type: DataFrame): the divided data that doesn't match the assigned feature's threshold
    """
    mask = np.asarray([i <= threshold for i in data[feature]])
    left = data[mask]
    right = data[~mask]
    
    return left, right


# [Note] You have to save the value of "ans_left" into the output file
left, right = make_partition(input_data, 'age', 61.0)
ans_left = left.shape[0]
print("ans_left = ", ans_left)

ans_left =  10


## Step5 : Build Decision Tree
Use the above functions to implement the decision tree

Instructions: 
1.    If current depth < max_depth and the remaining number of samples > min_samples_split: continue to classify those samples
2.    Use function *find_best_split()* to find the best split combination
3.    If the obtained information gain is **greater than 0**: can build a deeper decision tree (add depth)
4. Use function *make_partition()* to split the data into two parts
5. Save the features and corresponding thresholds (starting from the root) used by the decision tree into *ans_features[]* and *ans_thresholds[]* respectively




In [206]:
def build_tree(data, max_depth, min_samples_split, depth):
    """
    This function will build the decision tree
    args:
    * data(type: DataFrame): the data you want to apply to the decision tree
    * max_depth: the maximum depth of a decision tree
    * min_samples_split: the minimum number of instances required to do partition
    * depth: the height of the current decision tree
    return:
    * subtree: the decision tree structure including root, branch, and leaf (with the attributes and thresholds)
    """

    # check the condition of current depth and the remaining number of samples
    if depth < max_depth and len(data) >= min_samples_split:
        # call find_best_split() to find the best combination
        ig, threshold, feature = find_best_split(data)
        # check the value of information gain is greater than 0 or not 
        if ig > 0:
            # update the depth
            depth = depth + 1
            # call make_partition() to split the data into two parts
            left, right = make_partition(data, feature, threshold)
            # If there is no data split to the left tree OR no data split to the left tree
            if len(left) == 0 or len(right) == 0:
                # return the label of the majority
                label = (int)(len(data[data.get("diabetes_mellitus") == 1]) >= len(data[data.get("diabetes_mellitus") == 0]))
                return label
            else:
                question = "{} {} {}".format(feature, "<=", threshold)
                subtree = {question: []}

                # call function build_tree() to recursively build the left subtree and right subtree
                left_subtree = build_tree(left, max_depth, min_samples_split, depth)
                right_subtree = build_tree(right, max_depth, min_samples_split, depth)
                if left_subtree == right_subtree:
                    subtree = left_subtree
                else:
                    subtree[question].append(left_subtree)
                    subtree[question].append(right_subtree)
        else:
            # return the label of the majority
            label = (int)(len(data[data.get("diabetes_mellitus") == 1]) >= len(data[data.get("diabetes_mellitus") == 0]))
            return label
    else:
        # return the label of the majority
        label = (int)(len(data[data.get("diabetes_mellitus") == 1]) >= len(data[data.get("diabetes_mellitus") == 0]))
        return label

    return subtree

An example of the output from *build_tree()* 
```
{'bmi <= 33.5': [1, {'age <= 68.5': [0, 1]}]}
```
Therefore, 
```
ans_features = ['bmi', 'age']
ans_thresholds = [33.5, 68.5]
```



In [207]:
ans_features = []
ans_thresholds = []

decisionTree = build_tree(input_data, max_depth, min_samples_split, depth)
decisionTree

{'glucose_apache <= 207.0': [{'heart_rate_apache <= 133.0': [0, 1]}, 1]}

In [208]:
def read_tree(tree):
    global ans_features, ans_thresholds

    if type(tree) != dict:
        return 
    for entry in tree:
        ans_features.append(entry.split()[0])
        ans_thresholds.append((float)(entry.split()[2]))
        read_tree(tree[entry][0])
        read_tree(tree[entry][1])
read_tree(decisionTree)

In [209]:
# [Note] You have to save the features in the "decisionTree" structure (from root to branch and leaf) into the output file
ans_features

['glucose_apache', 'heart_rate_apache']

In [210]:
# [Note] You have to save the corresponding thresholds for the features in the "ans_features" list into the output file
ans_thresholds

[207.0, 133.0]

## Step6 : Save answers

In [211]:
basic = []
basic.append(ans_entropy)
basic.append(ans_informationGain)
basic.append(ans_ig)
basic.append(ans_value)
basic.append(ans_name)
basic.append(ans_left)
for i in range(len(ans_features)):
    basic.append(ans_features[i])
for m in range(len(ans_thresholds)):
    basic.append(ans_thresholds[m])

## Step7 : Split data
Split data into training set and validation set
> Note: We have split the data into training set and validation. You **cannot** change the distribution of the data.

In [212]:
num_train = 20
num_validation = 10

training_data = input_data.iloc[:num_train]
validation_data = input_data.iloc[-num_validation:]

y_train = training_data.get("diabetes_mellitus")
x_train = training_data.drop(['diabetes_mellitus'], axis=1)
y_validation = validation_data.get("diabetes_mellitus")
x_validation = validation_data.drop(['diabetes_mellitus'], axis=1)
y_validation = y_validation.values.flatten()

print(input_data.shape)
print(training_data.shape)
print(validation_data.shape)

(30, 10)
(20, 10)
(10, 10)


## Step8 to Step10 : Make predictions with a decision tree

Define the attributions of the decision tree
> You **cannot** modify the values of these attributes in this part

In [213]:
max_depth = 2
depth = 0
min_samples_split = 2
n_features = x_train.shape[1]

We have finished the function '*classify_data()*' below, however, you can modify this function if you prefer completing it on your own way.

In [214]:
def classify_data(instance, tree):
    """
    This function will predict/classify the input instance
    args:
    * instance: a instance(case) to be predicted
    return:
    * answer: the prediction result (the classification result)
    """
    equation = list(tree.keys())[0] 
    if equation.split()[1] == '<=':
        temp_feature = equation.split()[0]
        temp_threshold = equation.split()[2]
        if instance[temp_feature] > float(temp_threshold):
            answer = tree[equation][1]
        else:
            answer = tree[equation][0]
    else:
        if instance[equation.split()[0]] in (equation.split()[2]):
            answer = tree[equation][0]
        else:
            answer = tree[equation][1]

    if not isinstance(answer, dict):
        return answer
    else:
        return classify_data(instance, answer)


def make_prediction(tree, data):
    """
    This function will use your pre-trained decision tree to predict the labels of all instances in data
    args:
    * tree: the decision tree
    * data: the data to predict
    return:
    * y_prediction: the predictions
    """
    
    # [Note] You can call the function classify_data() to predict the label of each instance
    y_prediction = np.asarray([classify_data(data.iloc[i], tree) for i in range(len(data))])
    
    return y_prediction


def calculate_score(y_true, y_pred):
    """
    This function will calculate the f1-score of the predictions
    args:
    * y_true: the ground truth
    * y_pred: the predictions
    return:
    * score: the f1-score
    """
    T_value, T_count = np.unique(y_true[y_pred == 1], return_counts = True)
    P_value, P_count = np.unique(y_pred[y_true == 1], return_counts = True)
    precision = T_count[np.where(T_value == 1)[0][0]] / (np.sum(T_count))
    recall = P_count[np.where(P_value == 1)[0][0]] / (np.sum(P_count))

    score = 2 * (precision * recall) / (precision + recall)
    
    return score

In [215]:
decision_tree = build_tree(training_data, max_depth, min_samples_split, depth)

y_pred = make_prediction(decision_tree, x_validation)

# [Note] You have to save the value of "ans_f1score" the your output file
ans_f1score = calculate_score(y_validation, y_pred)
print("ans_f1score = ", ans_f1score)

ans_f1score =  0.6666666666666666


## Step11 : Write the Output File
Save all of your answers in a csv file, named as **hw2_basic.csv**

In [216]:
ans_path = 'hw2_basic.csv'

# [Note] You have to save the value of "ans_f1score" into the output file
basic.append(ans_f1score)
print(basic)

pd.DataFrame(basic).to_csv(ans_path, header = None, index = None)

[0.9871377743721863, 0.08345988684807149, 0.35229505124460436, 207.0, 'glucose_apache', 10, 'glucose_apache', 'heart_rate_apache', 207.0, 133.0, 0.6666666666666666]


# **Advanced Part** (35%)

## Step1: Load the input data
First, load the input file **hw2_input_advanced.csv**

In [217]:
advanced_data = pd.read_csv('hw2_input_advanced.csv')

You can split *advanced_data* into training set and validaiton set

In [218]:
num_train = (int)(len(advanced_data) * .9)
num_validation = len(advanced_data) - num_train

training_data = advanced_data.iloc[:num_train]
validation_data = advanced_data.iloc[-num_validation:]

y_train = training_data.get("diabetes_mellitus")
x_train = training_data.drop(['diabetes_mellitus'], axis=1)
y_validation = validation_data.get("diabetes_mellitus")
x_validation = validation_data.drop(['diabetes_mellitus'], axis=1)
y_validation = y_validation.values.flatten()

print(advanced_data.shape)
print(training_data.shape)
print(validation_data.shape)

(8379, 25)
(7541, 25)
(838, 25)


## Step2 : Load the test data
Load the input file **hw2_input_test.csv** to make predictions with the pre-trained random forest model

In [219]:
x_test = pd.read_csv('hw2_input_test.csv')
x_test

,age,bmi,gender,height,weight,arf_apache,bun_apache,creatinine_apache,gcs_eyes_apache,gcs_motor_apache,...,hematocrit_apache,intubated_apache,map_apache,resprate_apache,sodium_apache,temp_apache,ventilated_apache,wbc_apache,apache_4a_hospital_death_prob,apache_4a_icu_death_prob
0,62,32.866392,1,177.80,103.9,1,31.0,10.30,4,6,...,36.4,0,157,26,134,36.1,0,4.56,0.06,0.03
1,82,23.582766,0,157.50,58.5,0,26.0,0.54,3,4,...,32.8,0,42,25,142,36.1,0,6.00,0.14,0.06
2,61,31.684520,1,172.70,94.5,0,16.0,1.11,4,6,...,35.3,0,129,6,131,36.8,0,8.59,0.05,0.03
3,58,45.156250,0,160.00,115.6,0,19.0,0.70,1,4,...,30.1,1,131,23,138,34.9,1,16.03,0.33,0.22
4,74,25.817016,1,172.70,77.0,0,25.0,0.93,4,6,...,34.5,0,55,12,135,36.3,0,45.80,0.12,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,73,17.943584,0,157.48,44.5,0,12.0,0.30,4,6,...,33.8,0,129,9,144,36.9,0,7.70,0.02,0.01
836,79,29.049732,1,167.60,81.6,0,48.0,2.19,4,6,...,42.7,0,163,9,139,36.4,0,10.77,0.06,0.03
837,85,24.627827,0,152.40,57.2,0,11.0,0.48,3,5,...,29.5,0,67,9,139,36.6,0,7.35,0.16,0.05
838,68,32.510940,1,193.00,121.1,0,14.0,0.64,4,6,...,37.5,0,61,10,140,36.9,1,16.02,0.00,0.00


## Step3 : Build a Random Forest

Define the attributions of the random forest
> * You **can** modify the values of these attributes in advanced part
> * Each tree can have different attribute values
> * There must be **at least** 3 decision trees in the random forest model
> * Must use function *build_tree()* to build a random forest model
> * These are the parameters you can adjust : 


        ```
        max_depth = 
        depth = 0
        min_samples_split = 
        
        # total number of trees in a random forest
        n_trees = 

        # number of features to train a decision tree
        n_features = 

        # the ratio to select the number of instances
        sample_size = 
        n_samples = int(training_data.shape[0] * sample_size)
        ```




In [220]:
# Define the attributes
max_depth = 8
depth = 0
min_samples_split = 15

# total number of trees in a random forest
n_trees = 20

# number of features to train a decision tree
n_features = 20

# the ratio to select the number of instances
sample_size = 1 / n_trees
n_samples = int(training_data.shape[0] * sample_size)
print(n_samples)

377


In [221]:
def build_forest(data, n_trees, n_features, n_samples):
    """
    This function will build a random forest.
    args:
    * data: all data that can be used to train a random forest
    * n_trees: total number of tree
    * n_features: number of features
    * n_samples: number of instances
    return:
    * forest: a random forest with 'n_trees' of decision tree
    """

    # must reuse function build_tree()
    forest = np.asarray([build_tree(data.iloc[i * n_samples: (i + 1) * n_samples], max_depth, min_samples_split, depth) for i in range(n_trees)])
    return forest

### build_forest Usage
The section below shows the forest pretrained for the advanced part. For saving time, it is also possible to set:

 ``` createForest = False ``` 

in the section below. This will import a pretrained forest that is exactly the same to the forest generated with the current setting. 

In [222]:
createForest = False
if createForest:
    forest = build_forest(training_data, n_trees, n_features, n_samples)
else:
    forest = np.asarray([
        {'glucose_apache <= 216.0': [{'resprate_apache <= 15.0': [{'age <= 47.0': [{'glucose_apache <= 155.0': [{'bmi <= 33.26036394': [0, 1]}, 1]}, {'height <= 157.5': [1, {'map_apache <= 151.0': [{'arf_apache <= 0.0': [{'hematocrit_apache <= 24.5': [1, 0]}, 1]}, 1]}]}]}, {'hematocrit_apache <= 31.6': [{'arf_apache <= 0.0': [{'temp_apache <= 36.0': [{'temp_apache <= 34.6': [0, 1]}, {'age <= 61.0': [0, {'bmi <= 25.11637676': [0, {'bun_apache <= 14.0': [1, 0]}]}]}]}, 1]}, {'age <= 56.0': [0, {'temp_apache <= 36.6': [{'creatinine_apache <= 0.56': [1, {'hematocrit_apache <= 36.8': [{'resprate_apache <= 36.0': [1, 0]}, 0]}]}, 0]}]}]}]}, {'hematocrit_apache <= 18.6': [0, {'wbc_apache <= 8.0': [1, {'hematocrit_apache <= 33.3': [{'weight <= 71.5': [{'glucose_apache <= 259.0': [0, 1]}, 1]}, {'resprate_apache <= 36.0': [{'creatinine_apache <= 0.94': [{'age <= 28.0': [0, 1]}, {'gender <= 0': [0, 1]}]}, 0]}]}]}]}]},
        {'glucose_apache <= 193.0': [{'age <= 45.0': [0, {'glucose_apache <= 82.0': [{'age <= 57.0': [0, 1]}, {'hematocrit_apache <= 27.8': [{'bmi <= 30.68733944': [{'height <= 160.0': [0, {'bmi <= 24.30461788': [1, 0]}]}, 1]}, {'sodium_apache <= 136.0': [{'bun_apache <= 81.0': [0, 1]}, {'bmi <= 21.5530062': [0, {'glucose_apache <= 165.0': [{'creatinine_apache <= 0.71': [1, 0]}, 1]}]}]}]}]}]}, {'apache_4a_icu_death_prob <= 0.25': [{'height <= 157.4': [1, {'height <= 162.6': [{'bmi <= 22.48132907': [1, 0]}, {'bmi <= 32.96176435': [{'bmi <= 31.000062': [{'bmi <= 26.0086649': [{'bmi <= 25.30693205': [1, 0]}, 1]}, 0]}, 1]}]}]}, {'resprate_apache <= 33.0': [1, 0]}]}]},
        {'glucose_apache <= 177.0': [{'resprate_apache <= 33.0': [{'age <= 50.0': [{'glucose_apache <= 73.0': [1, 0]}, {'bmi <= 22.31392168': [0, {'creatinine_apache <= 1.96': [{'wbc_apache <= 4.6': [0, {'glucose_apache <= 85.0': [1, {'apache_4a_hospital_death_prob <= 0.09': [1, 0]}]}]}, 1]}]}]}, {'hematocrit_apache <= 37.0': [{'heart_rate_apache <= 128.0': [{'height <= 161.3': [0, {'heart_rate_apache <= 124.0': [{'creatinine_apache <= 1.1': [0, {'hematocrit_apache <= 32.4': [1, 0]}]}, 1]}]}, 0]}, 0]}]}, {'bmi <= 20.91960657': [0, {'sodium_apache <= 129.0': [0, {'hematocrit_apache <= 38.2': [{'bun_apache <= 86.0': [1, 0]}, {'hematocrit_apache <= 38.8': [0, {'heart_rate_apache <= 148.0': [{'resprate_apache <= 27.0': [{'bun_apache <= 14.0': [1, 0]}, 1]}, 0]}]}]}]}]}]},
        {'glucose_apache <= 161.0': [{'age <= 51.0': [0, {'bmi <= 29.50869237': [{'hematocrit_apache <= 36.4': [{'glucose_apache <= 67.0': [1, {'map_apache <= 72.0': [{'hematocrit_apache <= 31.1': [{'hematocrit_apache <= 26.0': [1, 0]}, 1]}, {'hematocrit_apache <= 35.8': [0, 1]}]}]}, 0]}, {'glucose_apache <= 106.0': [{'bmi <= 40.99600036': [{'weight <= 89.9': [1, {'sodium_apache <= 137.0': [0, 1]}]}, 1]}, {'hematocrit_apache <= 35.7': [{'wbc_apache <= 7.1': [0, {'age <= 77.0': [1, 0]}]}, 0]}]}]}]}, {'resprate_apache <= 44.0': [{'creatinine_apache <= 0.95': [{'apache_4a_hospital_death_prob <= 0.59': [1, 0]}, {'glucose_apache <= 211.0': [{'resprate_apache <= 32.0': [{'wbc_apache <= 8.5': [1, {'apache_4a_hospital_death_prob <= 0.09': [1, 0]}]}, 0]}, {'creatinine_apache <= 1.68': [{'resprate_apache <= 35.0': [{'wbc_apache <= 14.3': [{'age <= 54.0': [0, 1]}, 1]}, 0]}, 1]}]}]}, {'glucose_apache <= 227.0': [0, 1]}]}]},
        {'glucose_apache <= 187.0': [{'resprate_apache <= 33.0': [{'weight <= 62.59': [{'height <= 175.2': [0, 1]}, {'glucose_apache <= 67.0': [1, {'hematocrit_apache <= 35.9': [{'gcs_verbal_apache <= 2.0': [{'glucose_apache <= 175.0': [1, 0]}, {'glucose_apache <= 98.0': [{'height <= 177.8': [0, 1]}, 1]}]}, {'bmi <= 35.24409751': [0, 1]}]}]}]}, {'sodium_apache <= 137.0': [0, {'glucose_apache <= 123.0': [0, {'bmi <= 24.39286138': [0, {'wbc_apache <= 5.6': [0, 1]}]}]}]}]}, {'resprate_apache <= 41.0': [{'creatinine_apache <= 0.68': [1, {'height <= 180.0': [{'hematocrit_apache <= 16.6': [0, {'sodium_apache <= 129.0': [0, {'heart_rate_apache <= 148.0': [1, 0]}]}]}, {'apache_4a_hospital_death_prob <= 0.02': [1, {'map_apache <= 56.0': [1, 0]}]}]}]}, {'age <= 67.0': [0, {'resprate_apache <= 49.0': [0, 1]}]}]}]},
        {'glucose_apache <= 204.0': [{'temp_apache <= 36.1': [{'age <= 70.0': [{'temp_apache <= 35.9': [{'sodium_apache <= 136.0': [0, {'age <= 64.0': [1, 0]}]}, 1]}, 1]}, {'bmi <= 49.44368071': [{'hematocrit_apache <= 33.5': [{'arf_apache <= 0.0': [{'heart_rate_apache <= 108.0': [{'glucose_apache <= 85.0': [1, {'wbc_apache <= 6.7': [1, 0]}]}, {'creatinine_apache <= 0.58': [1, 0]}]}, 1]}, {'glucose_apache <= 143.0': [{'glucose_apache <= 68.0': [1, {'apache_4a_icu_death_prob <= 0.0': [{'bmi <= 27.75524582': [0, 1]}, 0]}]}, {'sodium_apache <= 136.0': [0, 1]}]}]}, 1]}]}, {'resprate_apache <= 37.0': [{'apache_4a_hospital_death_prob <= 0.18': [1, {'bmi <= 27.92026045': [0, 1]}]}, {'bun_apache <= 26.0': [0, 1]}]}]},
        {'glucose_apache <= 156.0': [{'heart_rate_apache <= 103.0': [{'hematocrit_apache <= 31.0': [{'heart_rate_apache <= 52.0': [0, {'glucose_apache <= 117.0': [{'bun_apache <= 37.0': [{'glucose_apache <= 75.0': [0, 1]}, 1]}, 0]}]}, {'glucose_apache <= 86.0': [{'height <= 170.0': [0, 1]}, {'heart_rate_apache <= 102.0': [{'resprate_apache <= 35.0': [0, {'resprate_apache <= 42.0': [1, 0]}]}, 1]}]}]}, {'weight <= 78.2': [{'weight <= 77.1': [{'bmi <= 21.44477937': [{'height <= 175.3': [1, 0]}, 0]}, 1]}, 0]}]}, {'weight <= 59.1': [{'bun_apache <= 23.0': [0, 1]}, {'resprate_apache <= 30.0': [{'temp_apache <= 37.5': [{'temp_apache <= 36.7': [{'creatinine_apache <= 0.94': [1, {'map_apache <= 55.0': [{'weight <= 130.8': [1, 0]}, 1]}]}, 1]}, 0]}, {'glucose_apache <= 262.0': [{'age <= 61.0': [{'height <= 147.3': [1, 0]}, {'bmi <= 24.46649361': [0, {'age <= 82.0': [1, 0]}]}]}, 1]}]}]}]},
        {'glucose_apache <= 151.0': [{'resprate_apache <= 31.0': [{'map_apache <= 57.0': [{'glucose_apache <= 112.0': [1, 0]}, {'bmi <= 23.47001488': [0, {'glucose_apache <= 111.0': [{'age <= 36.0': [0, {'bmi <= 32.89802498': [{'height <= 177.8': [0, 1]}, 1]}]}, {'bmi <= 23.53036634': [1, 0]}]}]}]}, {'age <= 63.0': [0, {'bun_apache <= 29.0': [{'apache_4a_hospital_death_prob <= 0.15': [0, 1]}, 0]}]}]}, {'heart_rate_apache <= 122.0': [{'glucose_apache <= 208.0': [{'wbc_apache <= 20.4': [{'map_apache <= 56.0': [{'bun_apache <= 11.0': [0, {'apache_4a_hospital_death_prob <= 0.43': [1, 0]}]}, {'heart_rate_apache <= 111.0': [{'map_apache <= 138.0': [{'heart_rate_apache <= 109.0': [0, 1]}, 1]}, 0]}]}, 0]}, {'map_apache <= 110.0': [{'age <= 30.0': [0, 1]}, {'bun_apache <= 43.0': [1, 0]}]}]}, {'glucose_apache <= 290.0': [{'weight <= 121.5': [{'apache_4a_hospital_death_prob <= 0.01': [1, 0]}, 1]}, 1]}]}]},
        {'glucose_apache <= 174.0': [{'age <= 49.0': [0, {'arf_apache <= 0.0': [{'bmi <= 30.64513371': [{'wbc_apache <= 6.4': [0, {'map_apache <= 120.0': [{'wbc_apache <= 23.1': [0, 1]}, 0]}]}, {'resprate_apache <= 30.0': [{'bun_apache <= 14.0': [0, {'gcs_eyes_apache <= 3.0': [0, 1]}]}, 0]}]}, 1]}]}, {'resprate_apache <= 15.0': [{'glucose_apache <= 492.0': [1, 0]}, {'hematocrit_apache <= 27.7': [{'glucose_apache <= 238.0': [0, 1]}, {'age <= 54.0': [1, {'hematocrit_apache <= 30.8': [1, {'creatinine_apache <= 0.72': [1, {'heart_rate_apache <= 140.0': [{'creatinine_apache <= 1.91': [1, 0]}, 1]}]}]}]}]}]}]},
        {'glucose_apache <= 167.0': [{'age <= 50.0': [{'height <= 157.0': [1, 0]}, {'hematocrit_apache <= 29.5': [{'gcs_verbal_apache <= 4.0': [1, {'bmi <= 24.46963971': [0, {'weight <= 83.7': [1, 0]}]}]}, {'glucose_apache <= 50.0': [1, {'gcs_verbal_apache <= 1.0': [0, {'bmi <= 27.30223048': [0, {'gcs_verbal_apache <= 4.0': [0, {'wbc_apache <= 6.9': [0, 1]}]}]}]}]}]}]}, {'resprate_apache <= 29.0': [{'apache_4a_hospital_death_prob <= 0.29': [{'wbc_apache <= 7.8': [1, {'sodium_apache <= 140.0': [{'glucose_apache <= 213.0': [{'map_apache <= 60.0': [1, 0]}, {'bun_apache <= 108.0': [1, 0]}]}, 1]}]}, 0]}, {'wbc_apache <= 9.9': [{'creatinine_apache <= 1.14': [{'bun_apache <= 7.0': [1, 0]}, {'bun_apache <= 36.0': [1, 0]}]}, {'hematocrit_apache <= 42.2': [{'apache_4a_icu_death_prob <= 0.07': [{'glucose_apache <= 180.0': [0, 1]}, {'bun_apache <= 23.0': [0, 1]}]}, 0]}]}]}]},
        {'glucose_apache <= 182.0': [{'age <= 52.0': [{'bmi <= 36.90194204': [{'sodium_apache <= 144.0': [0, 1]}, 0]}, {'gcs_verbal_apache <= 2.0': [{'apache_4a_hospital_death_prob <= 0.09': [1, {'bmi <= 24.60207612': [1, 0]}]}, {'bun_apache <= 36.0': [{'resprate_apache <= 28.0': [{'creatinine_apache <= 0.6': [1, {'sodium_apache <= 135.0': [0, {'bun_apache <= 19.0': [0, 1]}]}]}, {'creatinine_apache <= 0.99': [{'creatinine_apache <= 0.97': [{'glucose_apache <= 75.0': [1, 0]}, 1]}, 0]}]}, {'hematocrit_apache <= 32.2': [{'map_apache <= 55.0': [1, 0]}, 1]}]}]}]}, {'heart_rate_apache <= 129.0': [{'weight <= 145.8': [{'resprate_apache <= 25.0': [1, {'map_apache <= 137.0': [{'hematocrit_apache <= 34.4': [1, {'hematocrit_apache <= 37.5': [0, {'ventilated_apache <= 0.0': [1, 0]}]}]}, 0]}]}, 0]}, {'glucose_apache <= 219.0': [0, 1]}]}]},
        {'glucose_apache <= 179.0': [{'bmi <= 27.9550855': [{'hematocrit_apache <= 36.1': [{'wbc_apache <= 6.7': [0, {'sodium_apache <= 144.0': [{'temp_apache <= 36.7': [{'glucose_apache <= 124.0': [{'sodium_apache <= 131.0': [0, 1]}, 0]}, 0]}, 1]}]}, {'arf_apache <= 0.0': [0, 1]}]}, {'age <= 38.0': [0, {'glucose_apache <= 157.0': [{'gcs_verbal_apache <= 1.0': [{'bmi <= 30.58891139': [0, 1]}, {'apache_4a_hospital_death_prob <= 0.17': [{'sodium_apache <= 131.0': [1, {'map_apache <= 159.0': [0, 1]}]}, 0]}]}, {'map_apache <= 44.0': [0, 1]}]}]}]}, {'resprate_apache <= 38.0': [{'hematocrit_apache <= 37.1': [{'map_apache <= 60.0': [1, {'wbc_apache <= 18.4': [{'wbc_apache <= 5.6': [0, 1]}, 1]}]}, {'heart_rate_apache <= 126.0': [{'heart_rate_apache <= 60.0': [1, {'glucose_apache <= 284.0': [1, 0]}]}, 1]}]}, {'bun_apache <= 56.0': [{'glucose_apache <= 313.0': [{'wbc_apache <= 17.5': [{'bmi <= 24.85535139': [1, {'map_apache <= 109.0': [1, 0]}]}, 0]}, 1]}, 1]}]}]},
        {'glucose_apache <= 169.0': [{'hematocrit_apache <= 28.0': [{'ventilated_apache <= 0.0': [{'hematocrit_apache <= 27.2': [0, 1]}, {'resprate_apache <= 46.0': [{'bun_apache <= 54.0': [1, 0]}, 0]}]}, {'creatinine_apache <= 0.82': [0, {'age <= 66.0': [{'hematocrit_apache <= 35.5': [{'hematocrit_apache <= 33.2': [0, 1]}, 0]}, {'creatinine_apache <= 3.88': [{'glucose_apache <= 112.0': [{'bun_apache <= 40.0': [{'sodium_apache <= 140.0': [0, 1]}, 0]}, 0]}, 1]}]}]}]}, {'gender <= 0': [{'map_apache <= 57.0': [{'glucose_apache <= 175.0': [0, 1]}, {'heart_rate_apache <= 100.0': [{'heart_rate_apache <= 53.0': [1, 0]}, {'map_apache <= 110.0': [1, {'gcs_verbal_apache <= 1.0': [1, {'height <= 167.6': [0, 1]}]}]}]}]}, {'apache_4a_icu_death_prob <= 0.4': [{'gcs_verbal_apache <= 4.0': [1, {'sodium_apache <= 125.0': [0, {'creatinine_apache <= 0.97': [1, {'bun_apache <= 26.0': [1, {'apache_4a_icu_death_prob <= -1.0': [0, 1]}]}]}]}]}, 0]}]}]},
        {'glucose_apache <= 153.0': [{'age <= 52.0': [0, {'glucose_apache <= 84.0': [{'gender <= 0': [{'weight <= 51.0': [0, {'temp_apache <= 36.1': [1, {'height <= 149.0': [1, 0]}]}]}, 1]}, {'hematocrit_apache <= 26.7': [{'hematocrit_apache <= 25.3': [0, 1]}, {'sodium_apache <= 144.0': [{'gcs_verbal_apache <= 1.0': [0, {'resprate_apache <= 13.0': [{'wbc_apache <= 8.6': [1, 0]}, {'map_apache <= 177.0': [0, 1]}]}]}, 1]}]}]}]}, {'resprate_apache <= 44.0': [{'glucose_apache <= 188.0': [{'hematocrit_apache <= 31.9': [{'glucose_apache <= 182.0': [{'resprate_apache <= 39.0': [1, 0]}, 0]}, {'age <= 45.0': [1, {'sodium_apache <= 139.0': [0, 1]}]}]}, {'age <= 81.0': [{'resprate_apache <= 24.0': [1, {'bmi <= 36.47910884': [{'sodium_apache <= 145.0': [{'wbc_apache <= 5.5': [0, 1]}, 0]}, 1]}]}, 0]}]}, {'heart_rate_apache <= 107.0': [1, {'map_apache <= 44.0': [1, {'map_apache <= 148.0': [0, 1]}]}]}]}]},
        {'glucose_apache <= 187.0': [{'resprate_apache <= 14.0': [{'weight <= 65.86': [{'map_apache <= 47.0': [1, 0]}, {'wbc_apache <= 6.3': [1, {'glucose_apache <= 136.0': [{'temp_apache <= 36.8': [{'height <= 167.6': [0, {'hematocrit_apache <= 40.3': [1, 0]}]}, 1]}, 1]}]}]}, {'apache_4a_icu_death_prob <= 0.02': [0, {'resprate_apache <= 37.0': [{'wbc_apache <= 5.3': [1, {'hematocrit_apache <= 25.5': [0, {'temp_apache <= 35.8': [0, {'glucose_apache <= 138.0': [0, 1]}]}]}]}, {'weight <= 65.3': [0, {'ventilated_apache <= 0.0': [{'heart_rate_apache <= 126.0': [0, 1]}, 1]}]}]}]}]}, {'resprate_apache <= 34.0': [{'hematocrit_apache <= 41.7': [{'wbc_apache <= 15.5': [{'wbc_apache <= 4.2': [0, 1]}, {'ventilated_apache <= 0.0': [{'age <= 83.0': [1, 0]}, 0]}]}, 0]}, {'bmi <= 23.26702117': [0, {'glucose_apache <= 360.0': [{'temp_apache <= 36.2': [1, {'sodium_apache <= 139.0': [1, 0]}]}, 0]}]}]}]},
        {'glucose_apache <= 169.0': [{'resprate_apache <= 13.0': [{'glucose_apache <= 97.0': [{'weight <= 92.7': [{'hematocrit_apache <= 26.3': [1, 0]}, 1]}, {'bun_apache <= 38.0': [{'resprate_apache <= 5.0': [1, {'bmi <= 32.14124038': [{'temp_apache <= 36.9': [0, 1]}, 1]}]}, 1]}]}, {'gcs_verbal_apache <= 2.0': [{'wbc_apache <= 7.8': [0, {'height <= 167.6': [0, 1]}]}, {'heart_rate_apache <= 95.0': [{'glucose_apache <= 72.0': [1, 0]}, {'temp_apache <= 39.2': [{'temp_apache <= 36.0': [0, {'creatinine_apache <= 0.38': [1, 0]}]}, 1]}]}]}]}, {'glucose_apache <= 235.0': [{'resprate_apache <= 14.0': [{'bun_apache <= 7.0': [0, {'height <= 192.0': [1, 0]}]}, {'resprate_apache <= 23.0': [0, {'temp_apache <= 35.8': [1, {'heart_rate_apache <= 94.0': [1, {'apache_4a_hospital_death_prob <= 0.04': [0, {'glucose_apache <= 219.0': [1, 0]}]}]}]}]}]}, {'age <= 27.0': [0, {'bun_apache <= 97.0': [{'bmi <= 22.81877062': [1, {'bmi <= 32.38862053': [1, {'heart_rate_apache <= 100.0': [1, {'heart_rate_apache <= 108.0': [0, 1]}]}]}]}, 0]}]}]}]},
        {'glucose_apache <= 208.0': [{'resprate_apache <= 13.0': [{'resprate_apache <= 4.0': [0, {'age <= 32.0': [0, {'ventilated_apache <= 0.0': [{'creatinine_apache <= 0.53': [1, {'heart_rate_apache <= 56.0': [0, 1]}]}, 1]}]}]}, {'hematocrit_apache <= 32.9': [{'weight <= 58.9': [0, {'resprate_apache <= 18.0': [1, {'gcs_verbal_apache <= 3.0': [{'apache_4a_hospital_death_prob <= 0.42': [1, 0]}, {'map_apache <= 75.0': [{'age <= 59.0': [0, 1]}, 0]}]}]}]}, {'bmi <= 36.33000659': [{'glucose_apache <= 199.0': [0, 1]}, {'age <= 56.0': [{'temp_apache <= 36.9': [0, 1]}, 1]}]}]}]}, {'gcs_eyes_apache <= 2.0': [{'map_apache <= 171.0': [{'glucose_apache <= 392.0': [0, 1]}, 1]}, {'heart_rate_apache <= 140.0': [{'bmi <= 24.8000496': [{'bun_apache <= 28.0': [0, 1]}, {'sodium_apache <= 146.0': [{'weight <= 99.0': [1, {'glucose_apache <= 409.0': [1, 0]}]}, 0]}]}, 0]}]}]},
        {'glucose_apache <= 184.0': [{'hematocrit_apache <= 35.6': [{'weight <= 99.7': [{'age <= 32.0': [0, {'resprate_apache <= 37.0': [{'glucose_apache <= 150.0': [0, {'age <= 56.0': [0, 1]}]}, {'sodium_apache <= 144.0': [{'glucose_apache <= 57.0': [1, {'height <= 177.8': [0, 1]}]}, 1]}]}]}, {'gcs_verbal_apache <= 3.0': [1, {'height <= 162.5': [1, 0]}]}]}, {'age <= 60.0': [0, {'weight <= 77.1': [0, {'sodium_apache <= 137.0': [1, {'glucose_apache <= 74.0': [1, {'map_apache <= 130.0': [0, 1]}]}]}]}]}]}, {'heart_rate_apache <= 100.0': [1, {'temp_apache <= 37.3': [{'map_apache <= 108.0': [{'height <= 177.8': [{'sodium_apache <= 138.0': [{'sodium_apache <= 132.0': [1, {'gcs_motor_apache <= 5.0': [0, 1]}]}, 1]}, 1]}, {'temp_apache <= 36.6': [{'height <= 160.0': [1, {'age <= 47.0': [1, 0]}]}, 0]}]}, 1]}]}]},
        {'glucose_apache <= 160.0': [{'glucose_apache <= 73.0': [{'bmi <= 22.33004273': [0, 1]}, {'bmi <= 29.22650542': [{'age <= 59.0': [0, {'hematocrit_apache <= 21.2': [1, {'gcs_eyes_apache <= 3.0': [0, {'height <= 165.1': [0, {'height <= 170.0': [1, 0]}]}]}]}]}, {'resprate_apache <= 28.0': [{'age <= 49.0': [0, {'apache_4a_icu_death_prob <= 0.0': [0, 1]}]}, {'apache_4a_icu_death_prob <= 0.05': [{'bun_apache <= 4.0': [1, {'gcs_verbal_apache <= 1.0': [1, 0]}]}, 1]}]}]}]}, {'glucose_apache <= 209.0': [{'resprate_apache <= 14.0': [1, {'hematocrit_apache <= 24.6': [1, {'bmi <= 26.2565532': [{'creatinine_apache <= 1.93': [0, 1]}, {'age <= 79.0': [{'creatinine_apache <= 1.63': [1, 0]}, 0]}]}]}]}, {'wbc_apache <= 35.3': [{'bmi <= 42.82242197': [{'apache_4a_hospital_death_prob <= 0.26': [{'wbc_apache <= 5.11': [0, 1]}, 1]}, 1]}, 0]}]}]},
        {'glucose_apache <= 172.0': [{'glucose_apache <= 45.0': [1, {'resprate_apache <= 43.0': [{'bmi <= 34.45176132': [{'hematocrit_apache <= 40.9': [{'bun_apache <= 55.0': [{'wbc_apache <= 16.3': [{'hematocrit_apache <= 29.2': [1, 0]}, 0]}, 1]}, 0]}, {'bun_apache <= 48.0': [{'bun_apache <= 9.0': [0, {'weight <= 108.9': [1, {'weight <= 112.2': [0, 1]}]}]}, 0]}]}, 0]}]}, {'resprate_apache <= 13.0': [1, {'bmi <= 42.81203166': [{'apache_4a_icu_death_prob <= 0.0': [1, {'arf_apache <= 0.0': [{'sodium_apache <= 140.0': [{'glucose_apache <= 312.0': [{'creatinine_apache <= 0.49': [1, 0]}, 1]}, 1]}, 1]}]}, 0]}]}]}
    ])

## Step4 : Make predictions with the random forest
> Note: Please print the f1-score of the predictions of each decision tree

In [223]:
def make_prediction_forest(forest, data, mode = ""):
    """
    This function will use the pre-trained random forest to make the predictions
    args:
    * forest: the random forest
    * data: the data used to predict
    return:
    * y_prediction: the predicted results
    """
    y_result = np.asarray([(make_prediction(tree, data)) for tree in forest])
    if mode == "validation":
        for i in range(len(y_result)):
            print(f"Tree {i + 1} F1-Score = {calculate_score(y_validation, y_result[i])}")
    y_prediction = np.asarray([1 if np.mean(np.transpose(y_result)[i]) >= 0.5 else 0 for i in range(len(data))])

    return y_prediction

In [224]:
y_prediction = make_prediction_forest(forest, x_validation, "validation")
print(f"F1-Score = {calculate_score(y_validation, y_prediction)}")
y_pred_test = make_prediction_forest(forest, x_test)

Tree 1 F1-Score = 0.5829846582984658
Tree 2 F1-Score = 0.6274007682458387
Tree 3 F1-Score = 0.6437054631828979
Tree 4 F1-Score = 0.6728538283062645
Tree 5 F1-Score = 0.6538461538461539
Tree 6 F1-Score = 0.6358381502890174
Tree 7 F1-Score = 0.6491017964071857
Tree 8 F1-Score = 0.6044678055190539
Tree 9 F1-Score = 0.6500655307994757
Tree 10 F1-Score = 0.5967130214917825
Tree 11 F1-Score = 0.6292993630573248
Tree 12 F1-Score = 0.6633785450061652
Tree 13 F1-Score = 0.632
Tree 14 F1-Score = 0.6397058823529411
Tree 15 F1-Score = 0.6120906801007557
Tree 16 F1-Score = 0.6758104738154613
Tree 17 F1-Score = 0.6060606060606062
Tree 18 F1-Score = 0.6181818181818183
Tree 19 F1-Score = 0.6973532796317606
Tree 20 F1-Score = 0.6177215189873417
F1-Score = 0.7569528415961307


## Step5 : Write the Output File
Save your predictions from the **random forest** in a csv file, named as **hw2_advanced.csv**

In [225]:
advanced = []
for i in range(len(y_pred_test)):
    advanced.append(y_pred_test[i])

In [226]:
advanced_path = 'hw2_advanced.csv'
pd.DataFrame(advanced).to_csv(advanced_path, header = None, index = None)